In [1]:
using PowerSystems
using Dates
using TimeSeries
using DataFrames
using CSV
using DataStructures
using PowerSystemCaseBuilder
using PowerSimulations
using HiGHS

In [2]:
sys = System(100.0)

bus1 = ACBus(;
    number = 1,
    name = "bus1",
    bustype = ACBusTypes.REF,
    angle = 0.0,
    magnitude = 1.0,
    voltage_limits = (min = 0.9, max = 1.05),
    base_voltage = 230.0,
);

bus2 = ACBus(;
    number = 2,
    name = "bus2",
    bustype = ACBusTypes.PV,
    angle = 0.0,
    magnitude = 1.0,
    voltage_limits = (min = 0.9, max = 1.05),
    base_voltage = 230.0,
);

line = Line(;
    name = "line1",
    available = true,
    active_power_flow = 0.0,
    reactive_power_flow = 0.0,
    arc = Arc(; from = bus1, to = bus2),
    r = 0.00281, # Per-unit
    x = 0.0281, # Per-unit
    b = (from = 0.00356, to = 0.00356), # Per-unit
    rating = 2.0, # Line rating of 200 MVA / System base of 100 MVA
    angle_limits = (min = -0.7, max = 0.7),
);

load = PowerLoad(;
    name = "load1",
    available = true,
    bus = bus1,
    active_power = 0.0, # Per-unitized by device base_power
    reactive_power = 0.0, # Per-unitized by device base_power
    base_power = 100.0, # MVA
    max_active_power = 1.0, # 10 MW per-unitized by device base_power
    max_reactive_power = 0.0,
);

# https://nrel-sienna.github.io/PowerSystems.jl/stable/model_library/generated_RenewableDispatch/#RenewableDispatch
solar = RenewableDispatch(;
    name = "solar1",
    available = true,
    bus = bus2,
    active_power = 0.0, # Per-unitized by device base_power
    reactive_power = 0.0, # Per-unitized by device base_power
    rating = 1.0, # 5 MW per-unitized by device base_power
    prime_mover_type = PrimeMovers.PVe,
    reactive_power_limits = (min = 0.0, max = 0.05), # 0 MVAR to 0.25 MVAR per-unitized by device base_power
    power_factor = 1.0,
    operation_cost = RenewableGenerationCost(CostCurve(LinearCurve(0.0075))),
    # https://nrel-sienna.github.io/PowerSystems.jl/stable/model_library/renewable_generation_cost/#RenewableGenerationCost
    # https://nrel-sienna.github.io/PowerSystems.jl/stable/api/public/#InfrastructureSystems.CostCurve
    # https://nrel-sienna.github.io/PowerSystems.jl/stable/api/public/#InfrastructureSystems.LinearCurve
    base_power = 100, # MVA
);

add_components!(sys, [bus1, bus2, line, load, solar])

Try loading in PV+battery power output time series data from .csv files

In [3]:
data_dir = joinpath(homedir(), "ecen5407", "ecen5407_project2", "data",
"222628_32.73_-117.18_2012_pysam_output.csv")

solar_data = CSV.read(data_dir, DataFrame, header=1);

input_data = solar_data[:, "Column1"]

f(x; df = dateformat"yyyy-mm-dd HH:MM:SS.s") = Dates.DateTime(x, df)
datetimes = f.(input_data);

# System requires per-unitized time-series, normalizing for now but should eventually
# divide by actual system capacity
pv_max = maximum(solar_data[:,"ac_gross"])

solar_timearray = TimeArray(datetimes, solar_data[:,"ac_gross"]/pv_max);

solar_time_series = SingleTimeSeries(;
    name = "active_power",
    data = solar_timearray,
);

add_time_series!(sys, solar, solar_time_series);

Load data from Bri, slightly modified to exclude leap day and also normalized

In [4]:
data_dir = joinpath(homedir(), "ecen5407", "ecen5407_project2", "data",
"Project 2 - Load Profile_modified_index2.csv")

load_data = CSV.read(data_dir, DataFrame, header=1);

input_data = load_data[:, "Datetime"]

f(x; df = dateformat"yyyy-mm-dd HH:MM:SS.s") = Dates.DateTime(x, df)
datetimes = f.(input_data);

# This data is already "normalized" but could probably be more scrupulous about how
load_timearray = TimeArray(datetimes, load_data[:,"Load (MW)"]);

load_time_series = SingleTimeSeries(;
    name = "active_power",
    data = load_timearray,
);

add_time_series!(sys, load, load_time_series);


Then we take the timeseries and basically duplicate them to make a deterministic forecast timeseries - from https://nrel-sienna.github.io/PowerSystems.jl/v1.0/modeler_guide/time_series/#Transform-static-time-series-into-forecasts

In [8]:
transform_single_time_series!(
    sys,
    Dates.Hour(1), # horizon
    Dates.Minute(5), # interval
);

In [9]:
show_time_series(solar)

┌───────────────────────────────┬──────────────┬─────────────────────┬─────────────────────┬─────────┬───────────┬────────┬─────────────────────┐
│              time_series_type │         name │   initial_timestamp │          resolution │ horizon │  interval │  count │            features │
│                        String │       String │            DateTime │         Millisecond │    Hour │    Minute │  Int64 │   Dict{String, Any} │
├───────────────────────────────┼──────────────┼─────────────────────┼─────────────────────┼─────────┼───────────┼────────┼─────────────────────┤
│ DeterministicSingleTimeSeries │ active_power │ 2012-01-01T00:00:00 │ 300000 milliseconds │  1 hour │ 5 minutes │ 105397 │ Dict{String, Any}() │
└───────────────────────────────┴──────────────┴─────────────────────┴─────────────────────┴─────────┴───────────┴────────┴─────────────────────┘
┌──────────────────┬──────────────┬─────────────────────┬─────────────────────┬────────┬─────────────────────┐
│ time_series

In [10]:
show_time_series(load)

┌───────────────────────────────┬──────────────┬─────────────────────┬─────────────────────┬─────────┬───────────┬────────┬─────────────────────┐
│              time_series_type │         name │   initial_timestamp │          resolution │ horizon │  interval │  count │            features │
│                        String │       String │            DateTime │         Millisecond │    Hour │    Minute │  Int64 │   Dict{String, Any} │
├───────────────────────────────┼──────────────┼─────────────────────┼─────────────────────┼─────────┼───────────┼────────┼─────────────────────┤
│ DeterministicSingleTimeSeries │ active_power │ 2012-01-01T00:00:00 │ 300000 milliseconds │  1 hour │ 5 minutes │ 105397 │ Dict{String, Any}() │
└───────────────────────────────┴──────────────┴─────────────────────┴─────────────────────┴─────────┴───────────┴────────┴─────────────────────┘
┌──────────────────┬──────────────┬─────────────────────┬─────────────────────┬────────┬─────────────────────┐
│ time_series

From https://docs.juliahub.com/General/PowerSimulations/0.28.3/tutorials/decision_problem.html

In [11]:
solver = optimizer_with_attributes(HiGHS.Optimizer, "mip_rel_gap" => 0.5);
template_uc = template_unit_commitment();
set_device_model!(template_uc, Line, StaticBranch)
set_device_model!(template_uc, RenewableDispatch, RenewableFullDispatch)
set_device_model!(template_uc, PowerLoad, StaticPowerLoad)
set_network_model!(template_uc, NetworkModel(CopperPlatePowerModel))
problem = DecisionModel(template_uc, sys; optimizer = solver, name = "UC", horizon = Minute(5))
build!(problem, output_dir = mktempdir())

┌ Warning: Overwriting Line existing model
└ @ PowerSimulations /home/emco4286/.julia/packages/PowerSimulations/qdSY6/src/core/device_model.jl:116
┌ Warning: Overwriting RenewableDispatch existing model
└ @ PowerSimulations /home/emco4286/.julia/packages/PowerSimulations/qdSY6/src/core/device_model.jl:116
┌ Warning: Overwriting PowerLoad existing model
└ @ PowerSimulations /home/emco4286/.julia/packages/PowerSimulations/qdSY6/src/core/device_model.jl:116
┌ Error: DecisionModel Build Failed
│   exception =
│    ArgumentError: No matching metadata is stored. Tried Deterministic and DeterministicSingleTimeSeries.
│    Stacktrace:
│      [1] get_metadata(store::InfrastructureSystems.TimeSeriesMetadataStore, owner::PowerLoad, time_series_type::Type{Deterministic}, name::String; features::@Kwargs{})
│        @ InfrastructureSystems ~/.julia/packages/InfrastructureSystems/rXaFr/src/time_series_metadata_store.jl:423
│      [2] get_metadata
│        @ ~/.julia/packages/InfrastructureSystems/rXa

InfrastructureSystems.Optimization.ModelBuildStatusModule.ModelBuildStatus.FAILED = 1

Error: No matching metadata is stored. Ok, let's try to do their stupid metadata .json thing from https://nrel-sienna.github.io/PowerSystems.jl/stable/how_to/parse_ts_from_csvs/#Read-and-assign-time-series-to-System-using-these-parameters.

In [13]:
fname = joinpath(homedir(), "ecen5407", "ecen5407_project2", "data", "sienna_pointers.json")

"/home/emco4286/ecen5407/ecen5407_project2/data/sienna_pointers.json"

Try to add the metadata but it's not going to work

In [15]:
add_time_series!(sys, fname)

TaskFailedException: TaskFailedException

    nested task error: KeyError: key "solar1" not found
    Stacktrace:
     [1] getindex(h::Dict{String, Vector}, key::String)
       @ Base ./dict.jl:477
     [2] make_time_array(raw::InfrastructureSystems.RawTimeSeries, component_name::String, resolution::Millisecond)
       @ InfrastructureSystems ~/.julia/packages/InfrastructureSystems/rXaFr/src/time_series_parser.jl:286
     [3] make_time_array(info::InfrastructureSystems.TimeSeriesParsedInfo)
       @ InfrastructureSystems ~/.julia/packages/InfrastructureSystems/rXaFr/src/time_series_parser.jl:279
     [4] SingleTimeSeries(info::InfrastructureSystems.TimeSeriesParsedInfo)
       @ InfrastructureSystems ~/.julia/packages/InfrastructureSystems/rXaFr/src/single_time_series.jl:205
     [5] make_time_series!(cache::InfrastructureSystems.TimeSeriesParsingCache, ts_file_metadata::InfrastructureSystems.TimeSeriesFileMetadata)
       @ InfrastructureSystems ~/.julia/packages/InfrastructureSystems

Copied and pasted all this code to we can open the metadata file and see what's up

In [16]:
import JSON3
abstract type InfrastructureSystemsType end
abstract type InfrastructureSystemsComponent <: InfrastructureSystemsType end

mutable struct TimeSeriesFileMetadata
    "User description of simulation"
    simulation::AbstractString
    "String version of abstract type for the component associated with the time series."
    category::AbstractString
    "Calling module should determine the actual type."
    "Name of time_series component"
    component_name::AbstractString
    "User-defined name"
    name::AbstractString
    "Controls normalization of time series.
     Use 1.0 for pre-normalized data.
     Use 'Max' to divide the time series by the max value in the column.
     Use any float for a custom scaling factor."
    normalization_factor::Union{AbstractString, Float64}
    "Path to the time series data file"
    data_file::AbstractString
    "Resolution of the data being parsed in seconds"
    resolution::Dates.Period
    percentiles::Vector{Float64}
    time_series_type::DataType
    "Calling module must set."
    component::Union{Nothing, InfrastructureSystemsComponent}
    "Applicable when data are scaling factors. Accessor function on component to apply to
    values."
    scaling_factor_multiplier::Union{Nothing, AbstractString}
    scaling_factor_multiplier_module::Union{Nothing, AbstractString}
end

const g_cached_modules = Dict{String, Module}()

function get_module(module_name::AbstractString)
    cached_module = get(g_cached_modules, module_name, nothing)
    if !isnothing(cached_module)
        return cached_module
    end

    # root_module cannot find InfrastructureSystems if it hasn't been installed by the
    # user (but has been installed as a dependency to another package).
    mod = if module_name == "InfrastructureSystems"
        InfrastructureSystems
    else
        Base.root_module(Base.__toplevel__, Symbol(module_name))
    end

    g_cached_modules[module_name] = mod
    return mod
end


get_type_from_strings(module_name, type) =
    getproperty(get_module(module_name), Symbol(type))

function TimeSeriesFileMetadata(;
    simulation = "",
    category,
    component_name,
    name,
    normalization_factor,
    data_file,
    resolution,
    percentiles,
    time_series_type_module,
    time_series_type,
    scaling_factor_multiplier = nothing,
    scaling_factor_multiplier_module = nothing,
)
    return TimeSeriesFileMetadata(
        simulation,
        category,
        component_name,
        name,
        normalization_factor,
        data_file,
        resolution,
        percentiles,
        get_type_from_strings(time_series_type_module, time_series_type),
        nothing,
        scaling_factor_multiplier,
        scaling_factor_multiplier_module,
    )
end

metadata = open(fname) do io
    metadata = Vector{TimeSeriesFileMetadata}()
    data = JSON3.read(io, Array)
    for item in data
        parsed_resolution = Dates.Millisecond(Dates.Second(item["resolution"]))
        normalization_factor = item["normalization_factor"]
        if !isa(normalization_factor, AbstractString)
            normalization_factor = Float64(normalization_factor)
        end
        scaling_factor_multiplier =
            get(item, "scaling_factor_multiplier", nothing)
        scaling_factor_multiplier_module =
            get(item, "scaling_factor_multiplier_module", nothing)
        simulation = get(item, "simulation", "")
        push!(
            metadata,
            TimeSeriesFileMetadata(;
                simulation = simulation,
                category = item["category"],
                component_name = item["component_name"],
                name = item["name"],
                normalization_factor = normalization_factor,
                data_file = item["data_file"],
                resolution = parsed_resolution,
                # Use default values until CDM data is updated.
                percentiles = get(item, "percentiles", []),
                time_series_type_module = get(
                    item,
                    "module",
                    "InfrastructureSystems",
                ),
                time_series_type = get(item, "type", "SingleTimeSeries"),
                scaling_factor_multiplier = scaling_factor_multiplier,
                scaling_factor_multiplier_module = scaling_factor_multiplier_module,
            ),
        )
    end
    return metadata
end

2-element Vector{TimeSeriesFileMetadata}:
 TimeSeriesFileMetadata("DAY_AHEAD", "Generator", "solar1", "max_active_power", 1.0, "sienna_pv_gen.csv", Millisecond(300000), Float64[], SingleTimeSeries, nothing, "get_max_active_power", "PowerSystems")
 TimeSeriesFileMetadata("DAY_AHEAD", "ElectricLoad", "load1", "max_active_power", 1.0, "sienna_load.csv", Millisecond(300000), Float64[], SingleTimeSeries, nothing, "get_max_active_power", "PowerSystems")

In [19]:
metadata[1]

TimeSeriesFileMetadata("DAY_AHEAD", "Generator", "solar1", "max_active_power", 1.0, "sienna_pv_gen.csv", Millisecond(300000), Float64[], SingleTimeSeries, nothing, "get_max_active_power", "PowerSystems")

Ok so this seems to be working ok ... Troubleshooting below

In [ ]:
# https://github.com/NREL-Sienna/InfrastructureSystems.jl/blob/cd78bac21875e03341d838e574d5711aada97811/src/InfrastructureSystems.jl#L55


In [35]:
abstract type InfrastructureSystemsType end
abstract type InfrastructureSystemsContainer <: InfrastructureSystemsType end

const ADD_TIME_SERIES_BATCH_SIZE = 100

mutable struct TimeSeriesManager <: InfrastructureSystemsType
    data_store::TimeSeriesStorage
    metadata_store::TimeSeriesMetadataStore
    read_only::Bool
end

function TimeSeriesManager(;
    data_store = nothing,
    metadata_store = nothing,
    in_memory = false,
    read_only = false,
    directory = nothing,
    compression = CompressionSettings(),
)
    if isnothing(directory) && haskey(ENV, TIME_SERIES_DIRECTORY_ENV_VAR)
        directory = ENV[TIME_SERIES_DIRECTORY_ENV_VAR]
    end

    if isnothing(metadata_store)
        metadata_store = TimeSeriesMetadataStore()
    end

    if isnothing(data_store)
        data_store = make_time_series_storage(;
            in_memory = in_memory,
            directory = directory,
            compression = compression,
        )
    end
    return TimeSeriesManager(data_store, metadata_store, read_only)
end

function bulk_add_time_series!(
    mgr::TimeSeriesManager,
    associations;
    batch_size = ADD_TIME_SERIES_BATCH_SIZE,
)
    ts_keys = TimeSeriesKey[]
    batch = TimeSeriesAssociation[]
    sizehint!(batch, batch_size)
    open_store!(mgr.data_store, "r+") do
        for association in associations
            push!(batch, association)
            if length(batch) >= batch_size
                append!(ts_keys, add_time_series!(mgr, batch))
                empty!(batch)
            end
        end

        if !isempty(batch)
            append!(ts_keys, add_time_series!(mgr, batch))
        end
    end

    return ts_keys
end

function add_time_series!(mgr::TimeSeriesManager, batch::Vector{TimeSeriesAssociation})
    _throw_if_read_only(mgr)
    forecast_params = get_forecast_parameters(mgr.metadata_store)
    sts_params = StaticTimeSeriesParameters()
    num_metadata = length(batch)
    all_metadata = Vector{TimeSeriesMetadata}(undef, num_metadata)
    owners = Vector{TimeSeriesOwners}(undef, num_metadata)
    ts_keys = Vector{TimeSeriesKey}(undef, num_metadata)
    time_series_uuids = Dict{Base.UUID, TimeSeriesData}()
    metadata_identifiers = Set{Tuple}()
    TimerOutputs.@timeit_debug SYSTEM_TIMERS "add_time_series! in bulk" begin
        for (i, item) in enumerate(batch)
            throw_if_does_not_support_time_series(item.owner)
            check_time_series_data(item.time_series)
            metadata_type = time_series_data_to_metadata(typeof(item.time_series))
            metadata = metadata_type(item.time_series; item.features...)
            identifier = make_unique_owner_metadata_identifer(item.owner, metadata)
            if identifier in metadata_identifiers
                throw(ArgumentError("$identifier is present multiple times"))
            end
            push!(metadata_identifiers, identifier)
            if isnothing(forecast_params)
                forecast_params = _get_forecast_params(item.time_series)
            end
            check_params_compatibility(sts_params, forecast_params, item.time_series)
            all_metadata[i] = metadata
            owners[i] = item.owner
            ts_keys[i] = make_time_series_key(metadata)
            time_series_uuids[get_uuid(item.time_series)] = item.time_series
        end

        uuids = keys(time_series_uuids)
        existing_ts_uuids = if isempty(uuids)
            Base.UUID[]
        else
            list_existing_time_series_uuids(mgr.metadata_store, uuids)
        end
        new_ts_uuids = setdiff(keys(time_series_uuids), existing_ts_uuids)

        existing_metadata = list_existing_metadata(mgr.metadata_store, owners, all_metadata)
        if !isempty(existing_metadata)
            throw(
                ArgumentError(
                    "Time series data with duplicate attributes are already stored: " *
                    "$(existing_metadata)",
                ),
            )
        end
        for uuid in new_ts_uuids
            serialize_time_series!(mgr.data_store, time_series_uuids[uuid])
        end
        add_metadata!(mgr.metadata_store, owners, all_metadata)
    end
    return ts_keys
end


function add_time_series!(
    mgr::TimeSeriesManager,
    owner::TimeSeriesOwners,
    time_series::TimeSeriesData;
    features...,
)
    return add_time_series!(
        mgr,
        [TimeSeriesAssociation(owner, time_series; features...)],
    )[1]
end

function clear_time_series!(mgr::TimeSeriesManager)
    _throw_if_read_only(mgr)
    clear_metadata!(mgr.metadata_store)
    clear_time_series!(mgr.data_store)
end

function clear_time_series!(mgr::TimeSeriesManager, component::TimeSeriesOwners)
    _throw_if_read_only(mgr)
    for metadata in list_metadata(mgr.metadata_store, component)
        remove_time_series!(mgr, component, metadata)
    end
    @debug "Cleared time_series in $(summary(component))." _group =
        LOG_GROUP_TIME_SERIES
    return
end


abstract type InfrastructureSystemsComponent <: InfrastructureSystemsType end
const ComponentsByType = Dict{DataType, Dict{String, <:InfrastructureSystemsComponent}}

struct Components <: InfrastructureSystemsContainer
    data::ComponentsByType
    time_series_manager::TimeSeriesManager
    validation_descriptors::Vector
end

get_member_string(::Components) = "components"

function Components(
    time_series_manager::TimeSeriesManager,
    validation_descriptors = nothing,
)
    if isnothing(validation_descriptors)
        validation_descriptors = Vector()
    end

    return Components(ComponentsByType(), time_series_manager, validation_descriptors)
end

function serialize(components::Components)
    # time_series_storage and validation_descriptors are serialized elsewhere.
    return [serialize(x) for y in values(components.data) for x in values(y)]
end

function _add_component!(
    components::Components,
    component::T;
    skip_validation = false,
    allow_existing_time_series = false,
) where {T <: InfrastructureSystemsComponent}
    component_name = get_name(component)
    if !isconcretetype(T)
        throw(ArgumentError("add_component! only accepts concrete types"))
    end

    if !haskey(components.data, T)
        components.data[T] = Dict{String, T}()
    elseif haskey(components.data[T], component_name)
        throw(ArgumentError("$(component_name) is already stored for type $T"))
    end

    !skip_validation && check_component(components, component)

    if !allow_existing_time_series && has_time_series(component)
        throw(ArgumentError("cannot add a component with time_series: $component"))
    end

    components.data[T][component_name] = component
    return
end


const TIME_SERIES_STORAGE_FILE = "time_series_storage.h5"
const TIME_SERIES_DIRECTORY_ENV_VAR = "SIENNA_TIME_SERIES_DIRECTORY"
const VALIDATION_DESCRIPTOR_FILE = "validation_descriptors.json"
const SERIALIZATION_METADATA_KEY = "__serialization_metadata__"

"""
    mutable struct SystemData <: InfrastructureSystemsType
        components::Components
        "Masked components are attached to the system for overall management purposes but
        are not exposed in the standard library calls like [`get_components`](@ref).
        Examples are components in a subsystem."
        masked_components::Components
        validation_descriptors::Vector
        internal::InfrastructureSystemsInternal
    end

Container for system components and time series data
"""
mutable struct SystemData <: InfrastructureSystemsType
    components::Components
    masked_components::Components
    "Contains all attached component UUIDs, regular and masked."
    component_uuids::Dict{Base.UUID, <:InfrastructureSystemsComponent}
    "User-defined subystems. Components can be regular or masked."
    subsystems::Dict{String, Set{Base.UUID}}
    supplemental_attribute_manager::SupplementalAttributeManager
    time_series_manager::TimeSeriesManager
    validation_descriptors::Vector
    internal::InfrastructureSystemsInternal
end

"""
Construct SystemData to store components and time series data.

# Arguments

  - `validation_descriptor_file = nothing`: Optionally, a file defining component validation
    descriptors.
  - `time_series_in_memory = false`: Controls whether time series data is stored in memory or
    in a file.
  - `time_series_directory = nothing`: Controls what directory time series data is stored in.
    Default is the environment variable SIENNA_TIME_SERIES_DIRECTORY or tempdir() if that
    isn't set.
  - `compression = CompressionSettings()`: Controls compression of time series data.
"""
function SystemData(;
    validation_descriptor_file = nothing,
    time_series_in_memory = false,
    time_series_directory = nothing,
    compression = CompressionSettings(),
)
    validation_descriptors = if isnothing(validation_descriptor_file)
        []
    else
        read_validation_descriptor(validation_descriptor_file)
    end

    time_series_mgr = TimeSeriesManager(;
        in_memory = time_series_in_memory,
        directory = time_series_directory,
        compression = compression,
    )
    components = Components(time_series_mgr, validation_descriptors)
    supplemental_attribute_mgr = SupplementalAttributeManager()
    masked_components = Components(time_series_mgr, validation_descriptors)
    return SystemData(
        components,
        masked_components,
        Dict{Base.UUID, InfrastructureSystemsComponent}(),
        Dict{String, Set{Base.UUID}}(),
        supplemental_attribute_mgr,
        time_series_mgr,
        validation_descriptors,
        InfrastructureSystemsInternal(),
    )
end

function SystemData(
    validation_descriptors,
    time_series_manager,
    subsystems,
    supplemental_attribute_manager,
    internal,
)
    components = Components(time_series_manager, validation_descriptors)
    masked_components = Components(time_series_manager, validation_descriptors)
    return SystemData(
        components,
        masked_components,
        Dict{Base.UUID, InfrastructureSystemsComponent}(),
        subsystems,
        supplemental_attribute_manager,
        time_series_manager,
        validation_descriptors,
        internal,
    )
end

function open_time_series_store!(
    func::Function,
    data::SystemData,
    mode = "r",
    args...;
    kwargs...,
)
    open_store!(
        func,
        data.time_series_manager.data_store,
        mode,
        args...;
        kwargs...,
    )
end

function bulk_add_time_series!(
    data::SystemData,
    associations;
    batch_size = ADD_TIME_SERIES_BATCH_SIZE,
)
    bulk_add_time_series!(data.time_series_manager, associations; batch_size = batch_size)
end

function add_time_series_from_file_metadata!(
    data::SystemData,
    component_type::Type{<:InfrastructureSystemsComponent},
    file_metadata::Vector{TimeSeriesFileMetadata};
    resolution = nothing,
)
    return bulk_add_time_series!(
        data,
        _get_ts_associations_from_metadata(data, component_type, file_metadata, resolution),
    )
end

function _get_ts_associations_from_metadata(
    data::SystemData,
    component_type::Type{<:InfrastructureSystemsComponent},
    file_metadata,
    resolution,
)
    Channel() do channel
        cache = TimeSeriesParsingCache()
        for metadata in file_metadata
            if resolution === nothing || metadata.resolution == resolution
                for association in add_time_series_from_file_metadata_internal!(
                    data,
                    component_type,
                    cache,
                    metadata,
                )
                    put!(channel, association)
                end
            end
        end
    end
end


UndefVarError: UndefVarError: `TimeSeriesStorage` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

https://nrel-sienna.github.io/PowerSystems.jl/stable/how_to/parse_ts_from_csvs/

In [13]:
problem = DecisionModel(template_uc, sys; optimizer = solver, name = "UC", horizon = Minute(5))
build!(problem, output_dir = mktempdir())

┌ Error: DecisionModel Build Failed
│   exception =
│    ArgumentError: No matching metadata is stored. Tried Deterministic and DeterministicSingleTimeSeries.
│    Stacktrace:
│      [1] get_metadata(store::InfrastructureSystems.TimeSeriesMetadataStore, owner::PowerLoad, time_series_type::Type{Deterministic}, name::String; features::@Kwargs{})
│        @ InfrastructureSystems ~/.julia/packages/InfrastructureSystems/rXaFr/src/time_series_metadata_store.jl:423
│      [2] get_metadata
│        @ ~/.julia/packages/InfrastructureSystems/rXaFr/src/time_series_metadata_store.jl:386 [inlined]
│      [3] get_metadata
│        @ ~/.julia/packages/InfrastructureSystems/rXaFr/src/time_series_manager.jl:153 [inlined]
│      [4] get_time_series_metadata(::Type{Deterministic}, owner::PowerLoad, name::String; features::@Kwargs{})
│        @ InfrastructureSystems ~/.julia/packages/InfrastructureSystems/rXaFr/src/time_series_interface.jl:193
│      [5] get_time_series_metadata
│        @ ~/.julia/packag

InfrastructureSystems.Optimization.ModelBuildStatusModule.ModelBuildStatus.FAILED = 1